## Adafruit Mini GPS PA1010D GPS  
https://www.adafruit.com/product/4415

In [ ]:
import json
import pandas as pd

pd.set_option('max_colwidth', 200)

from meerkat import pa1010d, parser

In [ ]:
pd.__version__ #  tested with 1.1.3

#### Initialize Driver Class with Default I2C Address

In [ ]:
gps = pa1010d.PA1010D(bus_n=1)

In [ ]:
gps.csv_writer.device

#### NMEA Sentence Support  
The GPS module outputs several NMEA sentences, to collect all the data a continuous looping read is used until enough bytes are collected to reconstruct all the sentences. Like a UART connection, the module cycles through transmitting each supported NMEA sentence type. Therefore, by default all supported sentence types are returned.

The supported NMEA sentences are:  
'GGA', 'GSA', 'GSV', 'RMC', 'VTG'  
Refer to http://aprs.gids.nl/nmea/ and datasheet page 16 for details.

In [ ]:
gps.get()

Using the `nmea_sentences` keyword, specific NMEA sentences can be returned. Note that all sentences are still being transmitted on the I2C bus and being parsed by the driver.

In [ ]:
gps.get(nmea_sentences=['GGA', 'GSA'])

In [ ]:
gps.get(nmea_sentences=['GSV', 'RMC', 'VTG'])

#### CSV Writer Output

In [ ]:
gps.csv_writer.header

In [ ]:
gps.write(description="test_1", n=4, nmea_sentences=['GGA', 'GSA'], delay=1)

In [ ]:
gps.csv_writer.path

In [ ]:
m, df = parser.csv_resource(gps.csv_writer.path)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
def nmea_type(sentence):
    """Get NMEA sentence type"""
    s = sentence.split(",")
    return s[0][1:]

In [ ]:
df.nmea_sentence.apply(lambda x: nmea_type(sentence=x))

In [ ]:
def nmea_parse(sentence):
    """Parse NMEA sentence, removing start of sentence '$' and checksum delimiter '*' """
    start_seq = sentence[0]
    sentence = sentence[1:]
    sentence, checksum = sentence.split("*")
    return sentence.split(",") + [checksum]

In [ ]:
df.nmea_sentence.apply(nmea_parse)

In [ ]:
df["nmea_type"] = df.nmea_sentence.apply(lambda x: nmea_type(sentence=x))

In [ ]:
df

#### GGA sentence

Parse using DataFrame and list comprehension for the same result, but loses the row index values

In [ ]:
df_gga = pd.DataFrame([nmea_parse(gs) for gs in df.loc[df.nmea_type == "GNGGA", "nmea_sentence"].values])
df_gga.columns = parser.GGA_columns
df_gga

Parse using .loc and apply to preserve the source indexes

In [ ]:
df_gga = df.loc[df.nmea_type == "GNGGA"].apply(lambda x: nmea_parse(x.nmea_sentence), axis=1, result_type='expand')
df_gga.columns = parser.GGA_columns
df_gga

#### GSA sentence

In [ ]:
df_gsa = df.loc[df.nmea_type == "GPGSA"].apply(lambda x: nmea_parse(x.nmea_sentence), axis=1, result_type='expand')
df_gsa.columns = parser.GSA_columns
df_gsa

Merge metadata onto NMEA data

In [ ]:
df_meta = df.loc[df.nmea_type == "GNGGA", ["std_time_ms", "description", "sample_n", "datetime64_ns"]]
df_meta

In [ ]:
df_gga

In [ ]:
df_final = pd.concat([df_meta, df_gga], axis=1)
df_final

In [ ]:
df_final.dtypes

In [ ]:
def gps_dd(coord):
    """Convert ddmm.mmmm location to dd.dddd"""
    x = coord.split(".")
    head = x[0]
    minute_decimal = x[1]
    degree = head[0:-2]
    minute = head[-2:]
    return float(degree) + float(minute + "." + minute_decimal) / 60

In [ ]:
df_final.latitude.apply(gps_dd)

#### JSON Writer Output

In [ ]:
gps.json_writer.metadata_interval = 3
data = gps.publish(description="test_2", n=7, nmea_sentences=['GGA', 'RMC'], delay=2)
for n, d in enumerate(data):
    print(d)
    print("-"*40)

In [ ]:
# default writer format is CSV, switch to JSON
gps.writer_output = 'json'

In [ ]:
# writer method with description and sample number
gps.write(description='test_3', n=15, nmea_sentences=['GGA', 'GSA'])

In [ ]:
with open(gps.json_writer.path, 'r') as f:
    for _ in range(4):
        print(f.readline().strip())

In [ ]:
data = []
h = ""
with open(gps.json_writer.path, 'r') as f:
    for _ in range(15):
        s = f.readline().strip()
        js = json.loads(s)
        if "header" in js.keys():
            h = js["header"]
        data.append([js["std_time_ms"] , js["description"], js["sample_n"], js["nmea_sentence"]])

In [ ]:
pd.DataFrame(data, columns=h)